In [114]:
import configparser
from pathlib import Path
import pandas as pd
import pandas.io.sql as pandasql
from psycopg2 import connect
import numpy
import matplotlib as mpl
from IPython.display import HTML, display
import rick
import numpy as np
import matplotlib.pyplot as plt
from setuptools import setup, find_packages
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [115]:
merge_set = pd.read_sql('''select rank, start_vid, end_vid, length, link_set, new_start_vid, new_end_vid, new_length, new_link_set, st_hausdorffdistance  from test_route_ranks where length <100''',con)

,rank,start_vid,end_vid,length,link_set,new_start_vid,new_end_vid,new_length,new_link_set,st_hausdorffdistance
0,1,30362987,862292856,25.508114,[7980069200],30362987,30363018,82.921934,"[7980069200, 7980069210]",57.413820
1,2,30362987,862292856,25.508114,[7980069200],30362990,862292856,172.271724,"[295895040, 9456288900, 9456288910, 7980069200]",146.742578
2,1,30363018,862292856,57.413820,[7980069211],30363018,30362987,82.921934,"[7980069211, 7980069201]",25.496387
3,2,30363018,862292856,57.413820,[7980069211],30363091,862292856,649.817732,"[9456275601, 10558927991, 10558927981, 7545625...",592.355008
4,1,30363537,30415081,18.291170,[295878250],30363537,30415106,332.145852,"[295878250, 10558490460, 10558490470, 93632702...",309.076228
5,2,30363537,30415081,18.291170,[295878250],30363077,30415081,647.436875,"[11164223970, 11164223980, 9258577190, 2958792...",628.977869
6,1,30414528,30414578,15.958947,[295893650],30362990,30414578,648.810164,"[295895030, 295894770, 10566913110, 1056691312...",632.837526
7,2,30414528,30414578,15.958947,[295893650],30414528,30414915,849.570144,"[295893650, 295893590, 10566912430, 1056691244...",833.570593
8,1,30414531,30414534,20.142864,[295894790],30414531,30414916,866.959143,"[295894790, 295894750, 295894600, 295894250, 2...",846.760764
9,1,30414534,30414531,20.142864,[295894791],30414534,30362979,649.450279,"[295894791, 11591272571, 10566913661, 11591278...",629.148428


In [193]:
link_used = []
new_sets = []
unique_set = merge_set[['start_vid', 'end_vid']].drop_duplicates()

# loop through unique sets of intersection
# !!this will be ordered by priority and length eventually!!
for index, key in unique_set.reset_index(drop=True).iterrows():    
    start_vid = unique_set['start_vid'].iloc[index]
    end_vid = unique_set['end_vid'].iloc[index]
    print(start_vid, end_vid)
    prepare_set = merge_set[(merge_set['start_vid'] == start_vid) & (merge_set['end_vid'] == end_vid)]
    link_set = prepare_set['link_set'].iloc[0]
    
    # Check if this link_set was used to match another set before
    # if not, selection best and merge
    if link_used == [] or (np.array(link_set) != np.array(link_used)).all(axis=1).any():
        print('not used')
        # if there are more than one rank and length is less than 100 or something 
        
        if prepare_set['length'].iloc[0] < 100:    
            # if there are two possibility for this intersection set
            # then compare which one is better

            if prepare_set['rank'].shape[0] == 2: 
                rank1=prepare_set[(merge_set['rank']==1)]
                rank2=prepare_set[(merge_set['rank']==2)]

                # find hausdorffdistance difference and length difference
                hausorffdiff = abs(rank1['st_hausdorffdistance'].iloc[0]/rank2['st_hausdorffdistance'].iloc[0])
                lengthdiff = rank1['new_length'].iloc[0] - rank2['new_length'].iloc[0]

                # make selection based on hausdorffdistance difference and length difference
                # could try and play with the params a little bit
                if hausorffdiff < 5 and lengthdiff > 20:
                    select_set = 2
                else:
                    select_set = 1

                selection = merge_set[(merge_set['rank']==select_set)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]
                new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])
                
                # append new_links
                new_sets.append(new_set)
                
                # find links that were used to merge, append it to another list
                # also have to find out if it was using a set of link that was used before
                old_links = selection['link_set'].iloc[0]
                new_links = selection['new_link_set'].iloc[0]
                additional_links = list(set(new_links).difference(old_links))

                link_used.append(additional_links)
                
            # if there is only one possibility
            # see if that possibility is viable 
            elif prepare_set['rank'].shape[0] == 1: 
                print('only one!')
        
        # if the set length is longer than 100 then 
        # check to see if it was used to merge 
        # if yes dont append, if not append
        elif:        
                
                
                
    # if link_set as used, continue and dont append
    elif (np.array(link_set) == np.array(link_used)).all(axis=1).any():
        print(' used')    

30362987 862292856
not used
30363018 862292856
not used
30363537 30415081
not used
30414528 30414578


/etc/jupyterhub/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:25: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/etc/jupyterhub/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:26: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



AxisError: axis 1 is out of bounds for array of dimension 1

In [194]:
new_sets

[array([30362987, 30363018, 82.9219338709931,
        list([7980069200, 7980069210])], dtype=object),
 array([30363018, 30362987, 82.9219338709931,
        list([7980069211, 7980069201])], dtype=object),
 array([30363537, 30415106, 332.145851831287,
        list([295878250, 10558490460, 10558490470, 9363270280, 7545922850, 7545922860, 7545916030])],
       dtype=object)]